In [ ]:
# napari_ui.py
"""
Piccola interfaccia grafica (Qt) per caricare rapidamente uno stack di immagini RGB
in napari.

Dipendenze (crea prima un ambiente virtuale):
    pip install "napari[all]" PyQt6 scikit-image numpy

Esecuzione:
    python napari_ui.py

Packaging (macOS):
    # genera NapariRGBLoader.app dentro dist/
    pyinstaller --noconfirm --windowed --onefile \
               --name NapariRGBLoader napari_ui.py

"""

import sys
import glob
import numpy as np
from pathlib import Path
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout,
    QLineEdit, QPushButton, QFileDialog, QMessageBox
)
from PyQt6.QtCore import QSize
from skimage.io import imread_collection
import napari


class NapariLoader(QWidget):
    """Widget principale con barra percorso + pulsanti colorati."""

    def __init__(self):
        super().__init__()
        self.setWindowTitle("Napari RGB Loader")
        self.setFixedSize(420, 160)

        main_layout = QVBoxLayout(self)
        path_layout = QHBoxLayout()

        # campo di testo per il percorso
        self.path_edit = QLineEdit(self)
        self.path_edit.setPlaceholderText("Percorso cartella base (…/data1)")
        path_layout.addWidget(self.path_edit)

        # pulsante sfoglia
        browse_btn = self._colored_button("📂", "#4dabf7")
        browse_btn.clicked.connect(self.browse)
        path_layout.addWidget(browse_btn)

        main_layout.addLayout(path_layout)

        # pulsanti Avvia ed Esci
        btn_layout = QHBoxLayout()
        self.run_btn = self._colored_button("▶", "#51cf66")
        self.run_btn.clicked.connect(self.run_napari)
        btn_layout.addWidget(self.run_btn)

        quit_btn = self._colored_button("✕", "#ff6b6b")
        quit_btn.clicked.connect(self.close)
        btn_layout.addWidget(quit_btn)

        main_layout.addLayout(btn_layout)

    def _colored_button(self, text: str, color: str) -> QPushButton:
        btn = QPushButton(text)
        btn.setFixedSize(QSize(60, 60))
        btn.setStyleSheet(
            f"""
            QPushButton {{
                background-color: {color};
                border: none;
                color: white;
                font-size: 24px;
                border-radius: 6px;
            }}
            QPushButton:hover {{
                opacity: 0.85;
            }}
            """
        )
        return btn

    def browse(self):
        folder = QFileDialog.getExistingDirectory(
            self, "Seleziona cartella di lavoro"
        )
        if folder:
            self.path_edit.setText(folder)

    def run_napari(self):
        base_path = self.path_edit.text().strip()
        if not base_path:
            QMessageBox.warning(
                self, "Percorso mancante", "Inserisci o seleziona una cartella valida."
            )
            return

        rgb_path_glob = Path(base_path) / "RGBintegrals" / "layer_*.png"
        rgb_image_paths = sorted(glob.glob(str(rgb_path_glob)))
        if not rgb_image_paths:
            QMessageBox.warning(
                self,
                "Nessuna immagine trovata",
                f"Nessun file layer_*.png in {rgb_path_glob.parent}",
            )
            return

        # carica immagini
        rgb_images = imread_collection(rgb_image_paths).concatenate()

        # avvia napari
        viewer = napari.Viewer()
        viewer.add_image(rgb_images, name="RGB Integrals", rgb=True)
        viewer.add_labels(
            np.zeros(rgb_images.shape[:-1], dtype=int), name="Manual Labels"
        )
        napari.run()


def main():
    app = QApplication(sys.argv)
    window = NapariLoader()
    window.show()
    sys.exit(app.exec())


if __name__ == "__main__":
    main()


AttributeError: No napari attribute gui_qt